In [128]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import polars as pl
import streamlit as st
import json
import ast
import datetime as dt

# Polars EDA

In [111]:
df = pl.read_csv('../../metricbeabt-so-28042023.csv')

In [3]:
events = df['event'].unique()

In [4]:
df

@timestamp,@version,event,host,system
str,i64,str,str,str
"""2023-03-13T07:…",1,"""{'dataset': 's…","""{'name': 'QASE…","""{'cpu': {'core…"
"""2023-03-13T07:…",1,"""{'dataset': 's…","""{'name': 'QASE…","""{'cpu': {'core…"
"""2023-03-13T08:…",1,"""{'dataset': 's…","""{'name': 'QASE…","""{'cpu': {'core…"
"""2023-03-13T07:…",1,"""{'dataset': 's…","""{'name': 'QASE…","""{'cpu': {'core…"
"""2023-03-13T10:…",1,"""{'dataset': 's…","""{'name': 'QASE…","""{'cpu': {'core…"
"""2023-03-13T07:…",1,"""{'dataset': 's…","""{'name': 'QASE…","""{'cpu': {'core…"
"""2023-03-13T07:…",1,"""{'dataset': 's…","""{'name': 'QASE…","""{'cpu': {'core…"
"""2023-03-13T10:…",1,"""{'dataset': 's…","""{'name': 'QASE…","""{'cpu': {'core…"
"""2023-03-13T10:…",1,"""{'dataset': 's…","""{'name': 'QASE…","""{'cpu': {'core…"


In [5]:
for i in events:
    print(i)

{'dataset': 'system.network'}
{'dataset': 'system.memory'}
{'dataset': 'system.cpu'}


In [56]:
null_host = df.filter(df['system'].is_null()).select(pl.col('host'))
# null_host = null_host.unique()
# for i in null_host:
#     print(i)

In [57]:
null_host.groupby("host", maintain_order=True).agg(pl.count())

host,count
str,u32
"""{'name': 'QASE…",266653
"""{'name': 'PROD…",265771
"""{'name': 'DEVE…",266653
"""{'name': 'PREP…",266374
"""{'name': 'PRUE…",265979


In [40]:
df['system'].null_count()

1331430

In [45]:
df['system'].is_null()

system
bool
false
false
false
false
false
false
false
false
false


In [118]:
df = df.with_columns([pl.col('event').apply(ast.literal_eval),
                #  pl.col('system').apply(ast.literal_eval),
                 pl.col('host').apply(ast.literal_eval)],)

In [119]:
cpu = df.filter(pl.col('event') == '{"system.cpu"}')
network = df.filter(pl.col('event') == '{"system.network"}')
memory = df.filter(pl.col('event') == '{"system.memory"}')

In [120]:
cpu.__str__

<bound method DataFrame.__str__ of shape: (1_331_490, 5)
┌──────────────────────────┬──────────┬────────────────┬──────────────┬───────────────────────┐
│ @timestamp               ┆ @version ┆ event          ┆ host         ┆ system                │
│ ---                      ┆ ---      ┆ ---            ┆ ---          ┆ ---                   │
│ str                      ┆ i64      ┆ struct[1]      ┆ struct[1]    ┆ str                   │
╞══════════════════════════╪══════════╪════════════════╪══════════════╪═══════════════════════╡
│ 2023-03-13T07:04:31.079Z ┆ 1        ┆ {"system.cpu"} ┆ {"QASERVER"} ┆ {'cpu': {'cores': 4,  │
│                          ┆          ┆                ┆              ┆ 'idle': {'p…          │
│ 2023-03-13T07:13:31.085Z ┆ 1        ┆ {"system.cpu"} ┆ {"QASERVER"} ┆ {'cpu': {'cores': 4,  │
│                          ┆          ┆                ┆              ┆ 'idle': {'p…          │
│ 2023-03-13T08:16:41.112Z ┆ 1        ┆ {"system.cpu"} ┆ {"QASERVER"} ┆ {'cpu':

In [121]:
cpu = cpu.with_columns(pl.col('system').apply(json.loads))

ComputeError: JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [122]:
cpu

@timestamp,@version,event,host,system
str,i64,struct[1],struct[1],str
"""2023-03-13T07:…",1,"{""system.cpu""}","{""QASERVER""}","""{'cpu': {'core…"
"""2023-03-13T07:…",1,"{""system.cpu""}","{""QASERVER""}","""{'cpu': {'core…"
"""2023-03-13T08:…",1,"{""system.cpu""}","{""QASERVER""}","""{'cpu': {'core…"
"""2023-03-13T07:…",1,"{""system.cpu""}","{""QASERVER""}","""{'cpu': {'core…"
"""2023-03-13T10:…",1,"{""system.cpu""}","{""QASERVER""}","""{'cpu': {'core…"
"""2023-03-13T07:…",1,"{""system.cpu""}","{""QASERVER""}","""{'cpu': {'core…"
"""2023-03-13T07:…",1,"{""system.cpu""}","{""QASERVER""}","""{'cpu': {'core…"
"""2023-03-13T10:…",1,"{""system.cpu""}","{""QASERVER""}","""{'cpu': {'core…"
"""2023-03-13T10:…",1,"{""system.cpu""}","{""QASERVER""}","""{'cpu': {'core…"


In [33]:
df.with_columns(pl.col('event').apply(lambda x: dict(x)))

ComputeError: ValueError: dictionary update sequence element #0 has length 1; 2 is required

# Pickle EDA

In [125]:
cpu = pd.read_pickle('../../data/cpu.pkl')
network = pd.read_pickle('../../data/network.pkl')
memory = pd.read_pickle('../../data/memory.pkl')

cpu = pl.from_pandas(cpu)
network = pl.from_pandas(network)
memory = pl.from_pandas(memory)

In [126]:
cpu

cpu.cores,cpu.idle.pct,cpu.user.pct,cpu.system.pct,cpu.steal.pct,timestamp,event,node
i64,f64,f64,f64,f64,str,str,str
4,3.8109,0.0813,0.1078,null,"""2023-03-13T07:…","""system.cpu""","""QASERVER"""
4,3.7359,0.1422,0.1219,null,"""2023-03-13T07:…","""system.cpu""","""QASERVER"""
4,3.8563,0.0844,0.0593,null,"""2023-03-13T08:…","""system.cpu""","""QASERVER"""
4,3.9766,0.0078,0.0156,null,"""2023-03-13T07:…","""system.cpu""","""QASERVER"""
4,3.889,0.0688,0.0422,null,"""2023-03-13T10:…","""system.cpu""","""QASERVER"""
4,3.9266,0.0281,0.0453,null,"""2023-03-13T07:…","""system.cpu""","""QASERVER"""
4,3.9672,0.0187,0.0141,null,"""2023-03-13T07:…","""system.cpu""","""QASERVER"""
4,3.7546,0.1235,0.1219,null,"""2023-03-13T10:…","""system.cpu""","""QASERVER"""
4,3.8312,0.0641,0.1047,null,"""2023-03-13T10:…","""system.cpu""","""QASERVER"""


In [139]:
cpu = cpu.with_columns(pl.col('timestamp').apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ')))
network = network.with_columns(pl.col('timestamp').apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ')))
memory = memory.with_columns(pl.col('timestamp').apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%fZ')))

In [137]:
cpu.sort(pl.col('timestamp'))

cpu.cores,cpu.idle.pct,cpu.user.pct,cpu.system.pct,cpu.steal.pct,timestamp,event,node
i64,f64,f64,f64,f64,datetime[μs],str,str
4,3.9391,0.0203,0.0406,null,2023-03-01 00:00:00.303,"""system.cpu""","""QASERVER"""
4,3.9626,0.014,0.0234,null,2023-03-01 00:00:01.629,"""system.cpu""","""PREPRODUCCION"""
16,15.873,0.094,0.032,0.0,2023-03-01 00:00:05.906,"""system.cpu""","""PRUEBAS"""
8,7.7042,0.1018,0.194,null,2023-03-01 00:00:07.808,"""system.cpu""","""PRODUCCION"""
4,3.5039,0.2848,0.2112,null,2023-03-01 00:00:09.720,"""system.cpu""","""DEVELOP"""
4,3.8984,0.0579,0.0437,null,2023-03-01 00:00:10.315,"""system.cpu""","""QASERVER"""
4,3.8501,0.089,0.0609,null,2023-03-01 00:00:11.631,"""system.cpu""","""PREPRODUCCION"""
16,15.93,0.05,0.019,0.0,2023-03-01 00:00:15.906,"""system.cpu""","""PRUEBAS"""
8,7.7129,0.0952,0.1919,null,2023-03-01 00:00:17.823,"""system.cpu""","""PRODUCCION"""
